# SmartClaimX — Multi-Agent Health Insurance Claim Validator

SmartClaimX is an intelligent multi-agent system designed to automatically analyze, validate, and risk-score health insurance claims.
It simulates how real insurance companies evaluate claims by combining policy checking, fraud detection, medical validation, and cost estimation into one coordinated workflow.

This project uses Google's Agent Development Kit (ADK) with Gemini 2.0 models, multiple cooperating agents, and tool-based reasoning to deliver a complete end-to-end claim analysis pipeline.
SmartClaimX can take raw claim text, break it down into structured components, and produce a final decision:

✔ Approve
✔ Reject
✔ Review / Investigate Further

Each decision includes a clear explanation and per-agent outputs so users understand why the claim was evaluated that way.

The system is fully automated, modular, and can be extended to real insurance datasets in the future.

Concepts Used in SmartClaimX

1. Multi-Agent Architecture (A2A – Agent-to-Agent Workflow)

* SmartClaimX uses multiple specialized agents, each doing one specific task:
* PolicyCheckerAgent → checks policy rules and coverage
* FraudDetectorAgent → detects suspicious patterns
* MedicalValidationAgent → checks medical correctness using ICD mappings
* CostEstimatorAgent → extracts claimed amount & checks reasonability
* Coordinator Agent → orchestrates everything and returns final decision

2. Parallel Agents (Concurrent Reasoning)

3. Sequential Agent Pipeline

4. FunctionTool for Python Function Execution

5. InMemoryRunner for Local Simulation

6. Observability, Session Memory & Logging

7. GUI Interface Using ipywidgets

In [86]:
# Cells:
# 1 - Imports & ADK setup
# 2 - Observability & helpers (compaction, amount extraction)
# 3 - Local SessionService & MemoryBank
# 4 - FunctionTools (positional-only wrappers)
# 5 - Model factory + Sub-agents (Policy, Fraud, Medical, Cost)
# 6 - ParallelAgent (Fraud + Medical) and SequentialAgent example
# 7 - Coordinator (Supervisor) using AgentTool (A2A)
# 8 - Runner demo (run_debug) with sample claims
# 9 - ipywidgets GUI for interactive use

In [87]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_API_KEY")

# Imports & ADK setup

In [88]:
# ADK imports (from your notebooks)
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

# Standard libs
import re, json, logging, time, asyncio
from datetime import datetime
from collections import defaultdict
from pathlib import Path

# UI libs
try:
    import ipywidgets as widgets
    from IPython.display import display, Markdown, clear_output
except Exception:
    widgets = None

print("✅ ADK imports complete.")

✅ ADK imports complete.


# Observability, retry config, helpers

In [89]:
# Logging + metrics
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("SmartClaimX")
metrics = defaultdict(int)
def metric_inc(name, amt=1):
    metrics[name] += amt

# Retry options object (match your environment)
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429,500,503,504]
)

# helpers: context compaction and amount extraction
def compact_text(text, max_chars=1500):
    t = text.strip()
    return t if len(t) <= max_chars else t[:max_chars] + " ...[truncated]"

def extract_first_amount(text):
    # Find first numeric token (supports commas)
    m = re.search(r'(\d{1,3}(?:,\d{3})*(?:\.\d+)?|\d+)', text.replace('₹','').replace('$',''))
    if not m:
        return None
    s = m.group(1).replace(',','')
    try:
        return float(s)
    except:
        return None

logger.info("Observability and helpers ready.")

# Local SessionService & MemoryBank

In [90]:
# Simple local session service and memory bank for the notebook (not ADK sessions)
class InMemorySessionServiceLocal:
    def __init__(self):
        self.sessions = {}
    def create_session(self, session_id=None, user_id="anonymous", meta=None):
        sid = session_id or f"sid_{int(time.time()*1000)}"
        self.sessions[sid] = {"created": datetime.now(), "user_id": user_id, "meta": meta or {}, "history": []}
        return sid
    def append_message(self, session_id, role, text):
        if session_id not in self.sessions:
            self.create_session(session_id=session_id)
        self.sessions[session_id]["history"].append({"role": role, "text": text, "time": datetime.now()})
    def get_history(self, session_id):
        return self.sessions.get(session_id, {}).get("history", [])

class MemoryBankLocal:
    def __init__(self, max_entries=200):
        self.store = {}
        self.max_entries = max_entries
    def store_interaction(self, user_id, item):
        lst = self.store.setdefault(user_id, [])
        lst.append({**item, "time": datetime.now()})
        if len(lst) > self.max_entries:
            self.store[user_id] = lst[-self.max_entries:]
    def get_history(self, user_id, limit=10):
        return self.store.get(user_id, [])[-limit:]

session_service_local = InMemorySessionServiceLocal()
memory_bank_local = MemoryBankLocal()

logger.info("Local session service and memory bank instantiated.")

# FunctionTools (positional-only wrappers)

In [91]:
# Define Python helper functions and wrap them with FunctionTool using positional-only constructor.

def extract_amount_fn(claim_text: str):
    amt = extract_first_amount(claim_text)
    return {"amount": amt}

def icd_lookup_fn(diagnosis_text: str):
    # small mock ICD mapping
    icd_map = {
        "acl tear": "S83.5",
        "pneumonia": "J18.9",
        "appendicitis": "K35.3",
        "fracture femur": "S72.0"
    }
    key = diagnosis_text.lower()
    matches = {k:v for k,v in icd_map.items() if k in key}
    return {"matched": bool(matches), "matches": matches}

def policy_check_fn(claim_text: str, policy_id: str = "STD-001"):
    amount = extract_first_amount(claim_text) or 0
    covers = True
    reasons = []
    if "cosmetic" in claim_text.lower():
        covers = False
        reasons.append("Cosmetic procedures excluded.")
    if amount > 200000:
        reasons.append("Amount exceeds single-claim cap.")
    inpatient = any(word in claim_text.lower() for word in ["admitted","hospitalized","hospital stay"])
    return {"policy_id": policy_id, "covered": covers, "inpatient": inpatient, "amount": amount, "reasons": reasons}

# Wrap using the positional-only FunctionTool constructor that your ADK uses
extract_amount_tool = FunctionTool(extract_amount_fn)
icd_lookup_tool     = FunctionTool(icd_lookup_fn)
policy_check_tool   = FunctionTool(policy_check_fn)

logger.info("FunctionTools created: extract_amount_tool, icd_lookup_tool, policy_check_tool")

# Model factory + Sub-agents

In [92]:
# Model factory - try common signatures
def create_model():
    return Gemini(
        model="gemini-2.0-flash-lite-preview",  
        retry_options=retry_config
    )

# PolicyChecker Agent (uses policy_check_tool)
policy_agent = Agent(
    name="PolicyCheckerAgent",
    model=create_model(),
    instruction="""
You are PolicyCheckerAgent. Given a claim text, determine whether the claim is covered by a standard policy.
Return JSON exactly with keys:
{ "covered": true/false, "policy_id":"", "inpatient":true/false, "amount":number, "reasons":[ "..." ] }
""",
    tools=[policy_check_tool],
    output_key="policy_output"
)

# FraudDetector Agent
fraud_agent = Agent(
    name="FraudDetectorAgent",
    model=create_model(),
    instruction="""
You are FraudDetectorAgent. Analyze claim text for fraud indicators: improbable timelines, repeated claims, suspicious amounts, vague descriptions.
Return JSON:
{ "fraud_score": 0-100, "flags":[ "..."], "explain": "short text" }
""",
    output_key="fraud_output"
)

# MedicalValidation Agent (uses icd_lookup_tool)
med_agent = Agent(
    name="MedicalValidationAgent",
    model=create_model(),
    instruction="""
You are MedicalValidationAgent. Check diagnosis-treatment coherence and medical necessity.
Return JSON:
{ "medically_valid": true/false, "icd_matches": {...}, "notes":"short" }
""",
    tools=[icd_lookup_tool],
    output_key="medical_output"
)

# CostEstimator Agent (uses extract_amount_tool)
cost_agent = Agent(
    name="CostEstimatorAgent",
    model=create_model(),
    instruction="""
You are CostEstimatorAgent. Extract numeric amount and estimate if the claimed amount is reasonable (low/medium/high).
Return JSON:
{ "billing_issue": true/false, "estimated_range":"low/medium/high", "notes":"short" }
""",
    tools=[extract_amount_tool],
    output_key="cost_output"
)

logger.info("Sub-agents created: policy_agent, fraud_agent, med_agent, cost_agent.")

# ParallelAgent (Fraud + Medical) & SequentialAgent example

In [93]:
# Create a ParallelAgent according to your ADK pattern (name + sub_agents list)
parallel_fm = ParallelAgent(
    name="FraudAndMedicalParallel",
    sub_agents=[fraud_agent, med_agent]
)

# Example SequentialAgent (optional) - runs steps in order: parallel -> cost (illustrative)
sequential_demo = SequentialAgent(
    name="SequentialDemo",
    sub_agents=[parallel_fm, cost_agent]
)

logger.info("ParallelAgent and SequentialAgent objects created.")

# Coordinator (Supervisor) using AgentTool (A2A orchestration)

In [94]:
# --- SmartClaimX Coordinator (Strict JSON, With Decision Reasoning) ---

coordinator = Agent(
    name="SmartClaimXCoordinator",
    model=create_model(),
    instruction="""
You are SmartClaimXCoordinator.

Your job is to ALWAYS perform this exact workflow for every claim:

STEP 1 — CALL PolicyCheckerAgent  
• Use: PolicyCheckerAgent(request=<claim_text>)  
• Save result as policy_output.

STEP 2 — CALL FraudAndMedicalParallel  
• Runs FraudDetectorAgent + MedicalValidationAgent  
• Save results as fraud_output and medical_output.

STEP 3 — CALL CostEstimatorAgent  
• Use: CostEstimatorAgent(request=<claim_text>)  
• Save result as cost_output.

STEP 4 — AFTER ALL TOOL CALLS FINISH  
Combine everything and return ONLY this JSON:

{
  "policy_output": {},
  "fraud_output": {},
  "medical_output": {},
  "cost_output": {},
  "final_decision": "",
  "reasoning": ""
}

RULES:
- MUST call all 3 tools before creating the final JSON.
- MUST NOT return empty objects.
- MUST NOT skip tools.
- MUST NOT output markdown.
- MUST NOT output any explanation outside the JSON.
- reasoning = 2–3 concise lines.
""",
    tools=[
        AgentTool(policy_agent),
        AgentTool(parallel_fm),
        AgentTool(cost_agent)
    ],
    output_key="smartclaimx_result"
)

logger.info("Coordinator updated: forced tool calling enabled.")

# Runner creation & demo claims (run_debug)

In [ ]:
# Create runner (InMemoryRunner) and run sample claims using run_debug pattern
import os
os.environ["GOOGLE_API_KEY"] = secret_value_0

runner = InMemoryRunner(agent=coordinator)

claims = [
    {
        "id": "CLM-001",
        "user_id": "user_123",
        "text": "Patient admitted for ACL reconstruction surgery after slipping on wet floor. Claimed amount ₹85,000. Hospitalized for 3 days. Procedure at private hospital."
    },
    {
        "id": "CLM-002",
        "user_id": "user_456",
        "text": "Claim for cosmetic rhinoplasty. Claimed 150000. No hospitalization mentioned. Procedure labeled as 'cosmetic'."
    },
    {
        "id": "CLM-003",
        "user_id": "user_789",
        "text": "Multiple claims: last month claimed pneumonia treatment 12000, now again pneumonia claim 11500. Description vague, no tests attached."
    }
]

demo_results = {}

for c in claims:

    session_id = f"session_{c['id']}"
    user_id = c["user_id"]

    message = (
        f"ClaimID: {c['id']}\n"
        f"ClaimText: {compact_text(c['text'], 2000)}"
    )

    logger.info(f"\n🚀 Running SmartClaimX for Claim: {c['id']}")

    # === CORRECT ADK CALL ===
    resp = await runner.run_debug(
        user_messages=message,
        user_id=user_id,
        session_id=session_id
    )

    # Safety: Some ADK versions wrap outputs inside dict/event list
    # Normalizing result for readability
    readable = resp if isinstance(resp, str) else str(resp)

    demo_results[c['id']] = readable

    print("\n" + "="*60)
    print(f"                RESULT FOR {c['id']}")
    print("="*60 + "\n")
    print(readable)
    print("\n" + "="*60 + "\n")

    metric_inc("claims_processed")

logger.info("🏁 Demo run finished.")
logger.info("📊 Metrics: %s", dict(metrics))

# Interactive GUI (ipywidgets)

In [ ]:
# GUI for testing SmartClaimX with button click

import asyncio
from IPython.display import clear_output, display
import ipywidgets as widgets

# Text box for input
claim_input = widgets.Textarea(
    placeholder='Enter claim description here...',
    description='Claim:',
    layout=widgets.Layout(width='100%', height='120px')
)

# Button
run_button = widgets.Button(
    description='Run SmartClaimX',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

# Output display
output_box = widgets.Output()

# Async wrapper
async def process_claim_async(claim_text):
    session_id = "gui_test_session"
    user_id = "gui_user"

    message = f"ClaimID: GUI-TEST\nClaimText: {compact_text(claim_text, 2000)}"

    resp = await runner.run_debug(
        user_messages=message,
        user_id=user_id,
        session_id=session_id
    )

    return resp

# Button click handler
def on_button_click(b):
    output_box.clear_output()

    claim_text = claim_input.value.strip()
    if not claim_text:
        with output_box:
            print("⚠️ Please enter a claim text!")
        return

    with output_box:
        print("⏳ Processing claim...\n")

    # Run async task properly
    asyncio.create_task(run_and_display(claim_text))

# Helper async runner
async def run_and_display(text):
    resp = await process_claim_async(text)

    with output_box:
        clear_output()
        print("✅ Result:\n")
        print(resp)

# Bind click
run_button.on_click(on_button_click)

# Show GUI
display(claim_input, run_button, output_box)